In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

## DATA 

In [3]:
# Create data for parts 1 and 2
num_inliers = 20
num_outliers = 10
noise_ratio = 0.1
poly = np.random.uniform(size=[3, 1])  # random second-order polynomial
extremum = -poly[1]/(2*poly[0])
xstart = extremum - 0.5
lowest = np.polyval(poly, extremum)
highest = np.polyval(poly, xstart)
xspan = 1
yspan = highest - lowest
max_noise = noise_ratio * yspan
x = np.random.uniform(size=[1, num_inliers]) + xstart
y = np.polyval(poly, x)
y = y + (np.random.uniform(size=y.shape) - 0.5) * 2 * max_noise
data = np.concatenate([np.concatenate([x, np.random.uniform(size=[1, num_outliers]) + xstart], axis=1),
                       np.concatenate([y, np.random.uniform(size=[1, num_outliers]) * yspan + lowest], axis=1)], axis=0)


## Test 

In [4]:
data

array([[-2.07193124, -1.76128707, -1.83388825, -2.14951985, -1.37585074,
        -1.89441948, -2.08113942, -2.20614064, -1.55476885, -2.25940114,
        -2.28874346, -2.10296313, -2.16360444, -2.00413525, -1.41500451,
        -2.25832442, -1.67068032, -1.43542713, -1.38804362, -1.47716011,
        -1.67531432, -2.15653067, -2.14877685, -2.20467612, -1.72918703,
        -1.99838589, -2.19379577, -1.6661647 , -2.3286488 , -2.18133784],
       [ 0.0734929 ,  0.06427948,  0.06304356,  0.07472007,  0.0908886 ,
         0.06617433,  0.07525332,  0.08041555,  0.07793616,  0.08479783,
         0.09176866,  0.07299105,  0.07640322,  0.06586498,  0.08813207,
         0.08629945,  0.06870369,  0.08430908,  0.089543  ,  0.07894451,
         0.08472746,  0.09369629,  0.0714465 ,  0.07318128,  0.074111  ,
         0.07908002,  0.0796842 ,  0.0948442 ,  0.08625727,  0.08366252]])

In [5]:
data.shape

(2, 30)

In [7]:
indices = np.random.permutation(data.shape[1])[:3]

In [10]:
indices

array([ 2, 25, 11])

In [8]:
samples = data[:, indices]

In [11]:
samples

array([[-1.83388825, -1.99838589, -2.10296313],
       [ 0.06304356,  0.07908002,  0.07299105]])

In [9]:
guess = np.polyfit(samples[0, :], samples[1,:], 2)

In [12]:
guess

array([-0.57869392, -2.31520121, -2.23654451])

## Def parabolaRansac 

In [ ]:
import numpy as np

def parabolaRansac(data, max_noise):
    """
    best_guess_history is 3xnum_iterations with the polynome coefficients  
    from polyfit of the BEST GUESS SO FAR at each iteration columnwise and max_num_inliers_history is
    1xnum_iterations, with the inlier count of the BEST GUESS SO FAR at each iteration.
    """



## Code 

In [ ]:

# Part 1 - RANSAC with parabola model
best_guess_history, max_num_inliers_history = parabolaRansac(data, max_noise)


In [ ]:
# Compare with full data fit.
full_fit = np.polyfit(data[0, :], data[1, :], 2)

plt.clf()
plt.close()
fig, axs = plt.subplots(1, 2)

x = np.arange(xstart, xstart+1, 0.01)
for i in range(best_guess_history.shape[1]):
    guess_plot = axs[0].plot(x, np.polyval(best_guess_history[:, i], x), c='b', linewidth=1)

axs[0].plot(x, np.polyval(poly, x), 'g', linewidth=2, label='ground truth')
axs[0].plot(x, np.polyval(best_guess_history[:, -1], x), 'r', linewidth=2, label='RANSAC result')
axs[0].plot(x, np.polyval(full_fit, x), 'r--', linewidth=2, label='full data fit')
axs[0].set_title('RANSAC VS full fit')


line = Line2D([0], [0], color='b', linewidth=1, linestyle='-', label='RANSAC guesses')
handles, labels = axs[0].get_legend_handles_labels()
handles.append(line)
axs[0].legend(handles=handles, shadow=True, fancybox=True)

axs[1].plot(max_num_inliers_history)
axs[1].set_title('Max num inliers over iterations')
plt.show()

print('RMS of full fit =')
x = np.arange(0, 1, 0.01) + xstart
print(np.sqrt(np.mean(np.square(np.polyval(poly, x) - np.polyval(full_fit, x)))))
print('RMS of RANSAC =')
x = np.arange(0, 1, 0.01) + xstart
print(np.sqrt(np.mean(np.square(np.polyval(poly, x) - np.polyval(best_guess_history[:, -1], x)))))


## Solution 

In [6]:
import numpy as np

def parabolaRansac(data, max_noise):
    """
    best_guess_history is 3xnum_iterations with the polynome coefficients  
    from polyfit of the BEST GUESS SO FAR at each iteration columnwise and max_num_inliers_history is
    1xnum_iterations, with the inlier count of the BEST GUESS SO FAR at each iteration.
    """
    num_iterations = 100

    best_guess_history = np.zeros([3, num_iterations])
    max_num_inliers_history = np.zeros([num_iterations])

    best_guess = np.zeros([3, 1])
    max_num_inliers = 0
    rerun_on_inliers = True

    for i in range(num_iterations):
        # Model based on 3 samples:
        indices = np.random.permutation(data.shape[1])[:3]
        samples = data[:, indices]
        guess = np.polyfit(samples[0, :], samples[1,:], 2)

        # Evaluate amount of inliers
        errors = np.abs(np.polyval(guess, data[0, :]) - data[1,:])
        inliers = errors <= max_noise + 1e-5
        num_inliers = (inliers).sum()
        # Determine if the current guess is the best so far.
        if num_inliers > max_num_inliers:
            if rerun_on_inliers:
                guess = np.polyfit(data[0, inliers], data[1, inliers], 2)
            best_guess = guess
            max_num_inliers = num_inliers

        best_guess_history[:, i] = best_guess
        max_num_inliers_history[i] = max_num_inliers

    return best_guess_history, max_num_inliers_history